In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

In [2]:
# อ่านไฟล์ข้อมูลจากหลายไฟล์และรวมเข้าด้วยกัน
filePathList = [
    'TrecDataset/clean/cleanTrec01.csv',
    'TrecDataset/clean/cleanTrec02.csv',
    'TrecDataset/clean/cleanTrec03.csv',
    'TrecDataset/clean/cleanTrec04.csv'
]

In [3]:
# โหลดข้อมูลทั้งหมดและรวมเป็น DataFrame เดียว
dataFrames = [pd.read_csv(filePath) for filePath in filePathList]
allData = pd.concat(dataFrames, ignore_index=True)

In [4]:
# ตรวจสอบการกระจายตัวของข้อมูลก่อนการทำ undersampling
print("Original dataset label distribution:")
print(allData['label'].value_counts(normalize=True))
print(allData['label'].value_counts())

Original dataset label distribution:
label
1    0.665602
0    0.334398
Name: proportion, dtype: float64
label
1    50199
0    25220
Name: count, dtype: int64


In [5]:
# แยกข้อมูลตาม label
dataLabel0 = allData[allData['label'] == 0]
dataLabel1 = allData[allData['label'] == 1]

In [6]:
# ทำ undersampling ของ class ที่มีจำนวนมากกว่า (label 0 ในกรณีนี้)
if len(dataLabel0) > len(dataLabel1):
    dataLabel0 = resample(dataLabel0, 
                          replace=False,  # ไม่ทำซ้ำข้อมูล
                          n_samples=len(dataLabel1),  # ให้มีขนาดเท่ากับ class ที่น้อยกว่า
                          random_state=42)
else:
    dataLabel1 = resample(dataLabel1, 
                          replace=False, 
                          n_samples=len(dataLabel0), 
                          random_state=42)

In [7]:
# รวมข้อมูลที่ทำการ undersample แล้ว
balancedData = pd.concat([dataLabel0, dataLabel1], ignore_index=True)

In [8]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบในอัตราส่วน 70:30 โดยคงสัดส่วน label เท่าเดิม
trainData, testData = train_test_split(balancedData, test_size=0.3, random_state=42, stratify=balancedData['label'])

In [9]:
# แบ่ง trainData ออกเป็นสองส่วน (ครึ่งหนึ่ง) เพื่อบันทึกเป็นสองไฟล์
half_index = len(trainData) // 2
trainData01 = trainData.iloc[:half_index]
trainData02 = trainData.iloc[half_index:]

In [10]:
# ตรวจสอบการกระจายตัวของ label ในชุดฝึกและชุดทดสอบ
print("\nTraining set 01 label distribution:")
print(trainData01['label'].value_counts(normalize=True))
print(trainData01['label'].value_counts())

print("\nTraining set 02 label distribution:")
print(trainData02['label'].value_counts(normalize=True))
print(trainData02['label'].value_counts())

print("\nTest set label distribution:")
print(testData['label'].value_counts(normalize=True))
print(testData['label'].value_counts())


Training set 01 label distribution:
label
0    0.501699
1    0.498301
Name: proportion, dtype: float64
label
0    8857
1    8797
Name: count, dtype: int64

Training set 02 label distribution:
label
1    0.501699
0    0.498301
Name: proportion, dtype: float64
label
1    8857
0    8797
Name: count, dtype: int64

Test set label distribution:
label
1    0.5
0    0.5
Name: proportion, dtype: float64
label
1    7566
0    7566
Name: count, dtype: int64


In [11]:
# บันทึกไฟล์แยกออกเป็นสองไฟล์สำหรับ trainData และไฟล์สำหรับ testData
trainData01.to_csv('TrainData01.csv', index=False)
trainData02.to_csv('TrainData02.csv', index=False)
testData.to_csv('TestData.csv', index=False)